## Project: Estimation of accuracy of MOI for MVCs 
Created by: Thomas Hartka, MD, MSDS  
Date created: 1/27/22  
  
This notebook analyzes determines the sensitivity, specificity, and accuracy of the CDC MOI criteria for MVC in  patients.  This analysis is performed for existing criteria and new potential criteria.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import scipy.stats as st
import matplotlib.pyplot as plt
from itertools import combinations
import datetime
from multiprocessing import Process, Queue
import multiprocessing
import moi_analysis as moi

## Parameters

In [2]:
mvcs_data_file = "../Data/NASS_CISS-2000_2019-unfiltered.csv"
mvcs_imp_data_file = "../Data/NASS_CISS-2000_2019-imputated.csv"

In [3]:
use_imputed = True  # whether to use imputed or raw data
bootstrap_div = 5   # divisor of full sample len for bootstrapped sample
bootstrap_num = 1000   # number of bootstrapped samples

In [4]:
# column names
outcome = 'iss16'
existing_cit = ['int12occ','int18','ejection','other_death']
new_cit =['multicoll','abdeply','entrapment','unrestrained','rolled','rolled2','rolled4','sp45','sp65']
case_weights = 'casewgt'

## Read in data

In [5]:
if use_imputed:
    # use imputated data set
    mvcs = pd.read_csv(mvcs_imp_data_file)

else:
    # use unimputed data set
    mvcs = pd.read_csv(mvcs_data_file)

In [6]:
%%time
# add rollover critieria
mvcs['rolled2'] = mvcs.apply(lambda x: 1 if x['roll_turns'] >= 2 else 0, axis=1)
mvcs['rolled4'] = mvcs.apply(lambda x: 1 if x['roll_turns'] >= 4 else 0, axis=1)

# add posted speed limit critieria
mvcs['sp45'] = mvcs.apply(lambda x: 1 if x['splimit'] >= 72 else 0, axis=1)
mvcs['sp55'] = mvcs.apply(lambda x: 1 if x['splimit'] >= 89 else 0, axis=1)
mvcs['sp65'] = mvcs.apply(lambda x: 1 if x['splimit'] >= 105 else 0, axis=1)

# convert to unrestrained
mvcs['unrestrained'] = mvcs.apply(lambda x: 1 if x['any_restraint'] ==0 else 0, axis=1)

CPU times: user 1min 12s, sys: 3.67 s, total: 1min 16s
Wall time: 1min 16s


## Function for bootstrap evaluation of criteria

In [7]:
def bootstrap_str(dat, predictors, response, sample_size, bs_num, sig_dig=-1, verbose=False):
    '''
    This function runs bootstrapped acc, sens, and spec calculations.  It then finds the 
    median with 95% CI and returns the results as a string.
    
    Parameters:
        dat - data to analyze
        predictors - list of columns for predictors
        response - outcome column
        sample_size - size of bootstrapped sample
        bs_num - number of bootstrap iterations
    Returns:
        strings - median accuracy [95% CI], median sensitivity [95% CI], median specificity [95% CI]
    '''
    
    # lists for results
    acc = []
    sens = []
    spec = []
    
    for i in range(0,bs_num):
        # sample with replacement
        sample = dat.sample(sample_size, replace=True)

        # calculate AUC
        res = moi.var_perf(sample, predictors, response, 'casewgt')

        if verbose:
            if (i%100==0):
                print("Sample: ", i, " of ", bs_num)
        
        # store results to list
        acc += [res.accuracy]
        sens += [res.sensitivity]
        spec += [res.specificity]
        
    # get summary statistics
    acc_sum = moi.med_ci(acc, sig_dig=sig_dig)
    sens_sum = moi.med_ci(sens, sig_dig=sig_dig)
    spec_sum = moi.med_ci(spec, sig_dig=sig_dig)
    
    # convert to strings
    acc_str = str(acc_sum[0]) + " [" + str(acc_sum[1][0]) + "-" + str(acc_sum[1][1]) + "]"
    sens_str = str(sens_sum[0]) + " [" + str(sens_sum[1][0]) + "-" + str(sens_sum[1][1]) + "]"
    spec_str = str(spec_sum[0]) + " [" + str(spec_sum[1][0]) + "-" + str(spec_sum[1][1]) + "]"
    
    return acc_str, sens_str, spec_str

## Calculate performance of current criteria

In [8]:
# calculate size of each bootstrap sample
sample_size = round(len(mvcs)/bootstrap_div)

In [9]:
%%time
# data frame for results from current critieria
curr_res = pd.DataFrame(columns=['criterion','acc','sens','spec','acc_cum','sens_cum','spec_cum'])

# list of cumlative criteria
crit_cum = []

# loop through current criteria
for crit in existing_cit:

    # add variable to cumlative list
    crit_cum.append(crit)
    
    # evaluate individual and cumulative performance
    ind_res = bootstrap_str(mvcs, [crit], outcome, sample_size, bootstrap_num, sig_dig=2, verbose=False)
    cum_res = bootstrap_str(mvcs, crit_cum, outcome, sample_size, bootstrap_num, sig_dig=2, verbose=False)
    
    # create df with results from iteration
    itr_res = pd.DataFrame({'criterion':[crit], \
                            'acc':[ind_res[0]], \
                            'sens':[ind_res[1]], \
                            'spec':[ind_res[2]], \
                            'acc_cum':[cum_res[0]], \
                            'sens_cum':[cum_res[1]], \
                            'spec_cum':[cum_res[2]]})
    
    # add results to df
    curr_res = curr_res.append(itr_res)
                               
curr_res = curr_res.set_index('criterion')

display(curr_res)

,acc,sens,spec,acc_cum,sens_cum,spec_cum
criterion,,,,,,
int12occ,0.97 [0.97-0.98],0.2 [0.18-0.21],0.99 [0.99-0.99],0.97 [0.97-0.98],0.2 [0.18-0.21],0.99 [0.99-0.99]
int18,0.97 [0.97-0.98],0.23 [0.21-0.24],0.99 [0.99-0.99],0.97 [0.97-0.97],0.31 [0.3-0.33],0.98 [0.98-0.98]
ejection,0.98 [0.98-0.98],0.25 [0.23-0.26],0.99 [0.99-0.99],0.97 [0.96-0.97],0.48 [0.46-0.5],0.98 [0.97-0.98]
other_death,0.98 [0.98-0.98],0.2 [0.19-0.21],1.0 [1.0-1.0],0.97 [0.96-0.97],0.55 [0.52-0.57],0.97 [0.97-0.98]


CPU times: user 22min 35s, sys: 1min 48s, total: 24min 24s
Wall time: 23min 16s


## Calculate performance of new criteria

In [10]:
%%time

# data frame for results from current critieria
new_res = pd.DataFrame(columns=['criterion','acc','sens','spec','acc_cum','sens_cum','spec_cum'])

# list of cumlative criteria
crit_cum = []

# loop through current criteria
for crit in new_cit:

    print(crit)
    
    # add variable to cumlative list
    crit_cum = existing_cit + [crit]
    
    # evaluate individual and cumulative performance
    ind_res = bootstrap_str(mvcs, [crit], outcome, sample_size, bootstrap_num, sig_dig=2, verbose=False)
    cum_res = bootstrap_str(mvcs, crit_cum, outcome, sample_size, bootstrap_num, sig_dig=2, verbose=False)
    
    # create df with results from iteration
    itr_res = pd.DataFrame({'criterion':[crit], \
                            'acc':[ind_res[0]], \
                            'sens':[ind_res[1]], \
                            'spec':[ind_res[2]], \
                            'acc_cum':[cum_res[0]], \
                            'sens_cum':[cum_res[1]], \
                            'spec_cum':[cum_res[2]]})
    
    # add results to df
    new_res = new_res.append(itr_res)
                               
new_res = new_res.set_index('criterion')

display(new_res)

multicoll
abdeply
entrapment
unrestrained
rolled
rolled2
rolled4
sp45
sp65


,acc,sens,spec,acc_cum,sens_cum,spec_cum
criterion,,,,,,
multicoll,0.69 [0.69-0.7],0.59 [0.57-0.61],0.69 [0.69-0.7],0.69 [0.68-0.7],0.78 [0.76-0.8],0.69 [0.68-0.69]
abdeply,0.68 [0.67-0.69],0.54 [0.52-0.56],0.69 [0.68-0.69],0.67 [0.66-0.68],0.81 [0.79-0.83],0.67 [0.66-0.68]
entrapment,0.97 [0.97-0.97],0.22 [0.21-0.24],0.98 [0.98-0.99],0.96 [0.95-0.96],0.62 [0.6-0.64],0.96 [0.96-0.96]
unrestrained,0.83 [0.83-0.84],0.52 [0.5-0.54],0.84 [0.83-0.84],0.82 [0.81-0.83],0.75 [0.72-0.77],0.82 [0.82-0.83]
rolled,0.9 [0.89-0.9],0.3 [0.28-0.32],0.91 [0.9-0.91],0.89 [0.89-0.89],0.62 [0.59-0.64],0.89 [0.89-0.9]
rolled2,0.92 [0.91-0.92],0.27 [0.26-0.29],0.93 [0.92-0.93],0.91 [0.9-0.91],0.61 [0.59-0.63],0.91 [0.91-0.92]
rolled4,0.95 [0.95-0.95],0.19 [0.18-0.2],0.96 [0.96-0.97],0.94 [0.93-0.94],0.58 [0.56-0.6],0.94 [0.94-0.95]
sp45,0.53 [0.53-0.54],0.7 [0.68-0.72],0.53 [0.52-0.54],0.53 [0.52-0.54],0.84 [0.83-0.86],0.52 [0.51-0.53]
sp65,0.89 [0.89-0.9],0.18 [0.17-0.19],0.91 [0.9-0.91],0.88 [0.87-0.89],0.6 [0.57-0.62],0.89 [0.88-0.89]


CPU times: user 50min 10s, sys: 3min 25s, total: 53min 35s
Wall time: 51min 8s
